In [1]:
import os,shutil
import numpy as np
import pickle

class save_data(object):
    def __init__(self,data):
        self.data = data

with open("merge_data.pkl","rb") as f:
    data = pickle.load(f)
    
#dir, pktlen, payload len, flag, label
data_dir = {}

for item in data.data:

    label = item[0][-1]
    if label not in list(data_dir.keys()):
        data_dir[label] ={}
    
    for pkt in item:
        if pkt[0] == -1:
            continue
        lens = pkt[2] 
        if lens == 0:
            continue
        
        if lens not in list(data_dir[label].keys()):
            data_dir[label][lens] = 0
        data_dir[label][lens] += 1
        
data_dir_tuple = {}

for label in data_dir.keys():
    data_dir_tuple[label] = sorted(data_dir[label].items(),key=lambda item:item[1],reverse = True)
    
top20_dir = {}
for label in data_dir_tuple.keys():

    top20_dir[label] = [item[0] for item in data_dir_tuple[label][:20]]
    
data_dir_adj = {}
for flow in data.data:
    label = flow[0][-1]
    
    if label not in list(data_dir_adj.keys()):
        data_dir_adj[label] = []
    tmp_flow = [item[1] for item in flow]
    adj_flow = []
    for item in tmp_flow:
        idx_val = [abs(item-cand) for cand in top20_dir[label]]
        adj_flow.append(top20_dir[label][np.argmin(np.array(idx_val))])
    data_dir_adj[label].append(adj_flow)


#define a trans seq of 20  top20_dir data_dir_adj
trans_dir_20 = {}
seq_len = 2
for label in data_dir_adj:
    if label not in list(trans_dir_20.keys()):
        trans_dir_20[label] = np.zeros((seq_len,20,20))
    top_20 = [item for item in top20_dir[label] ]
    #initial point
    for flow in data_dir_adj[label]:
#         for i in range(seq_len):
#             if i >len(flow)-1:
#                 break
        idx = top_20.index(flow[0])
        trans_dir_20[label][0][idx][idx] += 1
    
#     trans_dir_20[label][0] = trans_dir_20[label][0]/np.diagonal(trans_dir_20[label][0]).sum()
    #trans matrix
    # split trans pairs
    print(top_20)
    for i in range(len(data_dir_adj[label])):
        flow = data_dir_adj[label][i]
        for j in range(1,len(flow)):
#             print(flow[j])
            trans_dir_20[label][1][top_20.index(flow[j-1])][top_20.index(flow[j])] += 1
#             print(top_20.index(flow[j-1]),top_20.index(flow[j]))
    
        
        
#dir, pktlen, payload len, flag, label
data_len_dir = {}

for item in data.data:

    label = item[0][-1]
    if label not in list(data_len_dir.keys()):
        data_len_dir[label] =[]
    
    data_len_dir[label].append(item)

    
def randomSeq(maxValue, num):

    maxValue = int(maxValue)
    suiji_ser = random.sample(range(1,maxValue), k=num-1) # 在1~maxValue之间，采集20个数据
    suiji_ser.append(0)   # 加上数据开头
    suiji_ser.append(maxValue)
    suiji_ser = sorted(suiji_ser)
    per_all_persons = [ suiji_ser[i]-suiji_ser[i-1] for i in range(1, len(suiji_ser)) ] # 列表推导式，计算列表中每两个数之间的间隔
    
    return per_all_persons


[490, 1460, 393, 409, 459, 248, 401, 517, 181, 126, 443, 498, 178, 168, 183, 177, 169, 167, 179, 170]
[93, 31, 216, 219, 163, 220, 761, 53, 56, 42, 38, 465, 691, 217, 389, 249, 87, 84, 491, 786]
[20, 901, 1013, 358, 69, 200, 949, 168, 53, 1460, 196, 75, 205, 36, 1237, 1253, 49, 7, 65, 1221]
[35, 31, 51, 517, 47, 82, 60, 71, 40, 37, 39, 32, 43, 38, 72, 15, 44, 41, 8, 4]
[1, 140, 44, 152, 38, 517, 45, 30, 46, 42, 34, 214, 200, 136, 87, 31, 43, 213, 28, 219]
[1460, 353, 35, 45, 422, 6, 31, 37, 473, 1350, 41, 946, 52, 64, 171, 53, 57, 190, 506, 302]
[38, 46, 286, 377, 1027, 1, 285, 190, 2882, 370, 376, 517, 1024, 287, 126, 141, 207, 1020, 1036, 142]
[459, 91, 194, 165, 181, 507, 197, 506, 508, 166, 245, 509, 150, 213, 196, 264, 421, 164, 188, 193]
[517, 93, 38, 99, 85, 126, 80, 78, 35, 51, 89, 128, 86, 84, 43, 96, 625, 69, 87, 90]
[1380, 441, 439, 1460, 67, 181, 1029, 1006, 93, 31, 1019, 126, 35, 1, 1447, 517, 51, 191, 1351, 1415]
[1460, 16413, 376, 344, 80, 68, 5840, 29200, 27740, 7300, 2

In [2]:
#handshake seq counting
handshake_seq_dir = {}
for label in list(data_len_dir.keys()):
    tmp = {}
    for flow in data_len_dir[label]:
#         print(item)
        if flow[0][-3] not in list(tmp.keys()):
            tmp[flow[0][-3]] = 0
        tmp[flow[0][-3]] += 1
    
    tmp = sorted(tmp.items(), key=lambda d:d[1], reverse = True)
    handshake_seq_dir[label] = tmp[0][0]

In [ ]:
#Enhanced Targeted attacking, two items should be provided : trans_array => trans_dir_20, adjusted length(headers) => top20_dir
import random,math
rej = ['FTPS','SFTP']
selected = ['360', 'aiqiyi', 'baidupan', 'huya', 'mgtv']
targeted_flows_dir_dir = {}#{src_clss0: {target_clss0: [], target_clss1" [] ....}, src_clss1: {target_clss0: [], target_clss1" [] ....} ...}
for src_clss in list(handshake_seq_dir.keys()):
    if src_clss in rej:
        continue
#     if src_clss not in selected:
#         continue
    targeted_flows_dir_dir[src_clss] = {}
    targeted_flows_dir = {} # {src_clss0: {target_clss0: [], target_clss1" [] ....}
    for target_clss in list(handshake_seq_dir.keys()):
        if src_clss == target_clss or target_clss in rej:
            continue
#         if target_clss not in selected:
#             continue
        targeted_flows_dir[target_clss] = []
        print("***** Refracting ", src_clss," ==> ",target_clss, " *****")
        # src_clss = '360'
        # target_clss = 'aiqiyi'

        # ack_pad = False

        can_flows = data_len_dir[src_clss]


        # org_flow_list_tmp = [flow for flow in can_flows]
        # org_flow_list = []
        # for item0 in org_flow_list_tmp:
        #     org_flow_list.append([item[1] for item in item0])

        payloads_blocks_list = []
        for flow in can_flows:
            #for each flow, first we gather the payload sequence blocks
            payloads_blocks = [] #(direction, start_idx, end_idx, flags(A,P), payload_len )
            i = 0
        #     print(flow)
            while i < len(flow)-1:
        #         print(flow[i])
        #     for i in range(len(flow)):
                if flow[i][2] == 0:
        #             payloads_blocks.append((flow[i][0],i,i,'A',0))
                    i += 1
                    continue
                start_idx = i
                payload_cul = []
                end_idx = 0
                #find the next opposite pkt
                for j in range(i,len(flow)):
                    if flow[j][0] != flow[i][0] and flow[j][2] !=0:

                        end_idx = j
        #                 payload_cul = np.array([item[2] for item in flow[i:j+1]]).sum()
        #                 payloads_block.append(flow[i][0],start_idx,end_idx,payload_cul)
                        break
                    end_idx = j
                payload_cul = np.array([item[2] for item in flow[i:j]]).sum()

        #         print(i,j)
                if end_idx == len(flow) -1:
                    payloads_blocks.append((flow[i][0],start_idx,end_idx,payload_cul))
                else:
                    payloads_blocks.append((flow[i][0],start_idx,end_idx-1,payload_cul))

                i = end_idx
            payloads_blocks_list.append(payloads_blocks)
        #     print(payloads_blocks)
        #         break
        #     break

        #payloads_blocks_list obtained, targeted attacking started
        target_trans_array = trans_dir_20[target_clss]
        target_adj_len_headers = top20_dir[target_clss]
        init_trans_array = np.diagonal(target_trans_array[0])
        fund_len = 54

        target_handshake_seq = handshake_seq_dir[target_clss]

        trans_flow_list = [] # pkt lens seq

        for k in range(len(can_flows)):
            flow = can_flows[k]
            hs_len = flow[0][-5]
            payloads_block = payloads_blocks_list[k]
            payload_cul = 0
            trans_idx = 0
            pre_lens_payload = 0
            trans_flow = []
            #append handshake pkts ([seq],start,end,'A/P')
            trans_flow.append(([target_handshake_seq[0]],0,0,'A'))
            trans_flow.append((([target_handshake_seq[2]],2,2,'A')))
        #     print(payloads_block)

        #     print(payloads_block)
            #select the initial point for the first block
            first_tuple = [(init_trans_array[x], x, target_adj_len_headers[x]) for x in range(len(init_trans_array))] #(count, idx, header)
            first_tuple_sorted = sorted(first_tuple, key = lambda x:x[0], reverse=True)

            #get the first positive direction block
            first_idx = None
            for g in range(len(payloads_block)):
                if payloads_block[g][0] == 1:
                    first_idx = g
                    break
        #         else:
        #             #if the first block is negative direction, append anyway in adj_flow
        #             for g1 in range(payloads_block[g][1],payloads_block[g][2]+1):
        #                 trans_flow.append(flow[g1][1])
            if first_idx == None: #error, append the original flow
        #         trans_flow.append([item[1] for item in flow])
                print("Error, no positive block detected!")
                print(flow[0][-4])             
                continue



            selected_first_length = None
            start_idx = payloads_block[first_idx][1]
            end_idx = payloads_block[first_idx][2]
            first_seq = []

            for lens in first_tuple_sorted:
                if payloads_block[first_idx][-1] >= lens[-1]:
                    selected_first_length = lens[-1]
                    break
            if selected_first_length != None:
                first_seq.append(fund_len + selected_first_length)
        #         trans_flow.append(fund_len)

                payload_cul += (payloads_block[first_idx][-1] - selected_first_length)
                pre_lens_payload = selected_first_length
                trans_idx += 1
            else:
                first_seq.append(fund_len + payloads_block[first_idx][-1])
        #         trans_flow.append(fund_len)

                payload_cul += (payloads_block[first_idx][-1] - payloads_block[first_idx][-1])
                pre_lens_payload = target_adj_len_headers[np.argmax(init_trans_array)]
                trans_idx += 1

            # first pkt of the first block has been re-organized, then we need to deal with the margin
#             print(trans_idx)
            ff_idx = list(target_adj_len_headers).index(pre_lens_payload)
            current_trans_array = target_trans_array[1][ff_idx]
            while payload_cul != 0:
                idx_tmp = list(target_adj_len_headers).index(pre_lens_payload)
        #         print(trans_idx,idx_tmp)
                #idx exceeds
                if trans_idx >= target_trans_array.shape[0]:
                    while payload_cul > 1460:
                        first_seq.append(1460+fund_len)
                        payload_cul -= 1460
                    first_seq.append(payload_cul+fund_len)
        #             trans_flow.append(fund_len)
                    payload_cul = 0
                    pre_lens_payload = target_adj_len_headers[np.argmax(current_trans_array)]
                    trans_idx += 1
                    break

                current_trans_array = target_trans_array[1][idx_tmp]
#                 print(current_trans_array)
                current_tuple = [(current_trans_array[x], x, target_adj_len_headers[x]) for x in range(len(current_trans_array))] #(count, idx, header)
                current_tuple_sorted = sorted(current_tuple, key = lambda x:x[0], reverse=True)
                
                
                #select the targeted lens
                select_idx = None
                for item in current_tuple_sorted:
                    if payload_cul >= item[-1]:
                        select_idx = item[1]
                        break
                
                if select_idx != None:
                    first_seq.append(target_adj_len_headers[select_idx]+fund_len)
        #             trans_flow.append(fund_len)

                    payload_cul -= target_adj_len_headers[select_idx]
                    pre_lens_payload = target_adj_len_headers[select_idx]
                    trans_idx =  (trans_idx + 1) % 20
#                     print(current_tuple_sorted)
#                     print("PaCul:", str(payload_cul), "idx: ", str(select_idx), "sel_len: ", str(target_adj_len_headers[select_idx]))
#                     print(select_idx)
#                     print(target_adj_len_headers[select_idx])
                else:# below the least value
        #             print(payload_cul)
                    first_seq.append(payload_cul+fund_len)
        #             trans_flow.append(fund_len)
                    payload_cul = 0
                    pre_lens_payload = target_adj_len_headers[np.argmax(current_trans_array)]
#                     print(pre_lens_payload)
                    trans_idx =  (trans_idx + 1) % 20
                    

            #append the first flow block to list
            trans_flow.append((first_seq,start_idx+hs_len,end_idx+hs_len,'P'))

            # deal with the following blocks
            for i in range(first_idx+1,len(payloads_block)):
#                 print(trans_idx)
#                 trans_idx = trans_idx % 20
                current_seq = []
        #         print(current_block)
                current_block = payloads_block[i]
                start_idx = current_block[1]
                end_idx = current_block[2]
                # negative direction blocks
                if current_block[0] == -1:
        #             for g1 in range(current_block[1],current_block[2]+1):
        #                 trans_flow.append(flow[g1][1])
                    continue

                # positive direction block, trans_idx out of consideration
                if current_block[0] == 1 and trans_idx >= target_trans_array.shape[0]:
                    for pkt in flow[current_block[1]:current_block[2]+1]:
                        if pkt[0] == 1:  
                            current_seq.append(pkt[2] + fund_len)
                    if current_block[-1] == 0:
                        trans_flow.append((current_seq,current_block[1]+hs_len,current_block[2]+hs_len,'A'))
                    else:
                        trans_flow.append((current_seq,current_block[1]+hs_len,current_block[2]+hs_len,'P'))
                    continue


                # positive direction blocks
                if current_block[0] == 1:

                    payload_cul = current_block[-1]

        #             print(current_block)
                    while payload_cul != 0:
                        # idx exceeds append randomly
                        if trans_idx >=target_trans_array.shape[0]:
                            if payload_cul == 0:
                                break
        #                     print(int(payload_cul/10))
                            pkt_number = random.randint(1,math.ceil(payload_cul/10))
                            seq = randomSeq(payload_cul, pkt_number)
                            for item in seq:
                                current_seq.append(item+fund_len)
        #                         trans_flow.append(fund_len)
                            payload_cul = 0
                            break

                        # normal idx
                        idx_tmp = list(target_adj_len_headers).index(pre_lens_payload)
                        current_trans_array = target_trans_array[1][idx_tmp]
                        # rank
                        current_tuple = [(current_trans_array[x], x, target_adj_len_headers[x]) for x in range(len(current_trans_array))] #(count, idx, header)
                        current_tuple_sorted = sorted(current_tuple, key = lambda x:x[0], reverse=True)
                        #select the targeted lens
                        select_idx = None
                        for item in current_tuple_sorted:
                            if payload_cul >= item[-1]:
                                select_idx = item[1]
                                break

                        if select_idx != None:
                            current_seq.append(target_adj_len_headers[select_idx]+fund_len)
        #                     trans_flow.append(fund_len)
                            payload_cul -= target_adj_len_headers[select_idx]
                            pre_lens_payload = target_adj_len_headers[select_idx]
#                             print(pre_lens_payload)
                            trans_idx =  (trans_idx + 1) % 20
                        else:# below the least value
                            current_seq.append(payload_cul+fund_len)
        #                     trans_flow.append(fund_len)
                            payload_cul = 0
                            pre_lens_payload = target_adj_len_headers[np.argmax(current_trans_array)]
#                             print(pre_lens_payload)
                            trans_idx =  (trans_idx + 1) % 20
                    trans_flow.append((current_seq,start_idx+hs_len,end_idx+hs_len,'P'))
                    continue

            #append save pkts
            #locate the fin idx
        #     if payloads_block[-1][0] != -1:
        #         fin_idx = None
        #         for i in range(len(flow)):
        #             if 'F' in flow[i][-2]:
        #                 fin_idx = i
        #                 break
        #         if fin_idx != None:
        #             for i in range(fin_idx,len(flow)):
        #                 trans_flow.append(flow[i][1])
        #     print(trans_flow)
            trans_flow_list.append((flow[0][-4], trans_flow))
#             print(trans_flow)
        #     print(trans_flow)
        #     print(flow)
        #     print(target_adj_len_headers)
#             break
        
        targeted_flows_dir[target_clss] = trans_flow_list
#         break
#     break
    targeted_flows_dir_dir[src_clss] = targeted_flows_dir



        
        

In [ ]:
import os,shutil
from scapy.all import *
from scapy.utils import PcapReader
import random
import copy
# reverse to the original flows
rej = ['FTPS','SFTP']
root = 'data'
root_dst = 'targeted_data'
root_adj_undir = 'data_targeted_undir'
root_adj_bidir = 'data_targeted_bidir'

root_adj_undir = os.path.join(root_dst,root_adj_undir)
root_adj_bidir = os.path.join(root_dst,root_adj_bidir)

if not os.path.exists(root_adj_undir):
    os.mkdir(root_adj_undir)

if not os.path.exists(root_adj_bidir):
    os.mkdir(root_adj_bidir)

for src_label in list(targeted_flows_dir_dir.keys()):
    if src_label in rej:
        continue
    src_label_path_undir = os.path.join(root_adj_undir, src_label)
    src_label_path_bidir = os.path.join(root_adj_bidir, src_label)
    
    targeted_flows_dir = targeted_flows_dir_dir[src_label]
    
    for label in list(targeted_flows_dir.keys()):
        if label in rej:
            continue
        dir_path = os.path.join(root,src_label)
        dir_path_targeted_undir = os.path.join(src_label_path_undir,label)
        dir_path_targeted_bidir = os.path.join(src_label_path_bidir,label)

        if not os.path.exists(dir_path_targeted_undir):
            os.makedirs(dir_path_targeted_undir)

        if not os.path.exists(dir_path_targeted_bidir):
            os.makedirs(dir_path_targeted_bidir)


        # flow level
        if label in ['SFTP','FTPS']:
            continue
        print("***** Reversing",src_label, " ==> " ,label," *****")
        for flow_pair in targeted_flows_dir[label]:
            double_break = False
            file = flow_pair[0]
            blocks = flow_pair[1] #tuple ([86, 81, 152, 67], 13, 14, 'P')
            if os.path.exists(os.path.join(dir_path_targeted_undir,file)):
                continue
            try:
#             if True:

                #open file 
                org_flow = rdpcap(os.path.join(dir_path,file))

                #confirm the src/dst ip
                src_ip = None
                src_port = None
                src_mac = None
                dst_ip = None
                dst_port = None
                dst_mac = None
                for ff in org_flow:
                    if ff['TCP'].flags == 'S':
                        src_ip = ff['IP'].src
                        dst_ip = ff['IP'].dst

                        src_port = ff['TCP'].sport
                        dst_port = ff['TCP'].dport

                        src_mac = ff['Ether'].src
                        dst_mac = ff['Ether'].dst     
                    if ff['TCP'].flags == 'SA':
                        src_ip = ff['IP'].dst
                        dst_ip = ff['IP'].src

                        src_port = ff['TCP'].dport
                        dst_port = ff['TCP'].sport

                        src_mac = ff['Ether'].dst
                        dst_mac = ff['Ether'].src
                        break
                if src_ip == None:
                    print('Error')
                    print(file)
                    break


                #idx reconstruction
                recon_blocks = []
                for i in range(len(blocks)-1):
                    
                    current_end_idx = blocks[i][-2]
                    next_start_idx = blocks[i+1][1]

                    recon_blocks.append((blocks[i],1))

                    if current_end_idx + 1 != next_start_idx:
                        recon_blocks.append(((current_end_idx+1,next_start_idx-1),-1))
                recon_blocks.append((blocks[-1],1))
                # deal with the remaining pkts
                if blocks[-1][-2] < len(org_flow)-1:
                    recon_blocks.append(((blocks[-1][-2]+1,len(org_flow)-1),-1))

                #traverse the recon_blocks
                # maintain a ack and a seq numbers
                idx = recon_blocks[0][1]
                ack = org_flow[idx]['TCP'].ack
                seq = org_flow[idx]['TCP'].seq

                # eastablish a standard pkt
                standard_pkt = None
                for pkt in org_flow:
                    if len(pkt) >=70:
                        standard_pkt = pkt
                if standard_pkt == None:
                    print("Standard pkt not found !")
                    print(file)
                    double_break = True
                    continue


                adj_flow_pcap_bidir = []
                adj_flow_pcap_undir = []

                for block_pair in recon_blocks:
                    if double_break == True:
                        break
                    block = block_pair[0]

                    #extract ack and seq number of the first pkt of each block
                    idx = block[1]
                    ack = org_flow[idx]['TCP'].ack
                    seq = org_flow[idx]['TCP'].seq




                    # positive direction blocks
                    if block_pair[-1] == 1:
                        #[start,end)
                        start_idx = block[1]
                        end_idx = block[2] + 1

                        if block[-1] == 'A': #block without payload
                            for i in range(start_idx,end_idx):
                                tmp_pkt = copy.deepcopy(org_flow[i])
                                adj_flow_pcap_bidir.append(tmp_pkt)
                                adj_flow_pcap_undir.append(tmp_pkt)

                            continue
                        if block[-1] == 'P': #block with payload
                            # cumulate payload from one block
                            payload_cul = bytearray()
                            for i in range(start_idx,end_idx):
                                if org_flow[i].haslayer("Raw"):
                                    payload_cul += bytearray(bytes(org_flow[i]['Raw']))
                            adj_seq = block[0]
                            #check out if the culmulate payload generated sequence equals to the original payload length
                            if np.array(adj_seq).sum() - np.array(adj_seq).shape[0]*54 != len(payload_cul):
                                print("Error! payloads length unequal!")
                                print(file)
                                print(block)
#                                 break
                            # construct the fundamental pkt
                            pkt_recon_fund = copy.deepcopy(standard_pkt)
                            pkt_recon_fund["Ether"].src = src_mac
                            pkt_recon_fund["Ether"].dst = dst_mac
                            pkt_recon_fund["IP"].dst = dst_ip
                            pkt_recon_fund["IP"].src = src_ip
                            pkt_recon_fund["TCP"].dport = dst_port
                            pkt_recon_fund["TCP"].sport = src_port
                            # ack = ack[0]
                            pkt_recon_fund["TCP"].ack = ack

                            #random time seq
                            time_start = org_flow[block[1]].time
                            time_end = org_flow[block[2]].time
                            duration = time_end - time_start
        #                     print(duration)
                            time_seq = [duration/(len(adj_seq)*1.0) for x in range(len(adj_seq))]
                            if len(time_seq) > 3:

                                time_seq[1] += time_seq[0]
                                time_seq[0] = 0
                                time_seq[-2] += time_seq[-1]
                                time_seq[-1] = 0

                            # construct each pkt
                            payload_used_cul = 0
                            for i in range(len(adj_seq)):
                                gen_pkt_len = adj_seq[i]
                                gen_pkt_time = time_seq[i]
                                payload_used = gen_pkt_len - fund_len

                                # copy from fundamental pkt
                                pkt_recon = copy.deepcopy(pkt_recon_fund)
                                # seq[i] = seq[0] + payload_used_cul
                                pkt_recon["TCP"].seq = seq + payload_used_cul
                                pkt_recon.time = time_start + gen_pkt_time
                                time_start += gen_pkt_time
                                pkt_recon["TCP"].flags = "PA" #PSH ACK

                                # concat payload payload_cul
                                im_py = payload_cul[payload_used_cul:payload_used_cul+payload_used]
        #                         print(repr(pkt_recon))
                                pkt_recon['Raw'] = Packet(im_py)

                                payload_used_cul += payload_used

                                #convert pkt to btye flow
                                pkt_recon.len = len(pkt_recon)
                                pkt_recon['IP'].id += i
                                pkt_recon['IP'].len = (len(pkt_recon) -14)
        #                         print(len(pkt_recon))
                                #append to new flow
                                adj_flow_pcap_bidir.append(pkt_recon)
                                adj_flow_pcap_undir.append(pkt_recon)


                    # negative direction blocks
                    if block_pair[-1] == -1:
                        start_idx = block[0]
                        end_idx = block[1] + 1
                        for i in range(start_idx,end_idx):
                            adj_flow_pcap_bidir.append(org_flow[i])
                        #[start,end)
            except:
                print("Error")
                print(file)
                continue
            if double_break:
                continue
            output_path_targeted_undir = os.path.join(dir_path_targeted_undir,file)
            output_path_targeted_bidir = os.path.join(dir_path_targeted_bidir,file)



            #write the undirectional targeted flow
            writer = PcapWriter(output_path_targeted_undir)
            for pkt in adj_flow_pcap_undir:
                writer.write(pkt)
            writer.close()

            #write the bidirectional targeted flow
#             writer = PcapWriter(output_path_targeted_bidir)
#             for pkt in adj_flow_pcap_bidir:
#                 writer.write(pkt)
#             writer.close()

                
                


            
            
        
        
        
        
#         org_undir_flow = []